In [17]:
#!/usr/bin/env python3

import argparse
import json
import os
import importlib.util

from src.learner.raytune_learner import TuneWrapper as StimulusTuneWrapper
from launch_utils import import_class_from_file, get_experiment
from src.utils.yaml_model_schema import YamlRayConfigLoader


model_path = "/home/luisasantus/Desktop/crg_cluster/projects/crossevo_stimulus/stimulus/fpkm_model/fpkm_dummy_model.py"
json_experiment = "/home/luisasantus/Desktop/crg_cluster/projects/crossevo_stimulus/stimulus/fpkm_model/fpkm_pipelinegenerated.json"
config_path = "/home/luisasantus/Desktop/crg_cluster/projects/crossevo_stimulus/stimulus/fpkm_model/fpkm_dummy_model.yaml"
data_path = "/home/luisasantus/Desktop/crg_cluster/projects/crossevo_stimulus/stimulus/work/9d/0ebf1c448bbe1a0ca653b26bea66f7/test-fpkm-#2-trans.csv"
best_config_path = "/home/luisasantus/Desktop/crg_cluster/projects/crossevo_stimulus/stimulus/fpkm_model/best_config.json"

# import the model correctly but do not initialize it yet, ray_tune does that itself
model_class = import_class_from_file(model_path)

# read json and retrieve experiment name and then initialize the experiment class
experiment_name = None
with open(json_experiment, 'r') as in_json:
    d = json.load(in_json)
    experiment_name = d["experiment"]
initialized_experiment_class = get_experiment(experiment_name)

# Create the learner
learner = StimulusTuneWrapper(config_path, model_class, data_path, initialized_experiment_class)


# Tune the model
learner.tune()
# save best config
#learner.store_best_config(best_config_path)



2024-05-06 17:58:31,589	ERROR tune_controller.py:1383 -- Trial task failed for trial TuneModel_76f62_00002
Traceback (most recent call last):
  File "/home/luisasantus/mambaforge/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/luisasantus/mambaforge/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/luisasantus/mambaforge/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/home/luisasantus/mambaforge/lib/python3.10/site-packages/ray/_private/worker.py", line 2563, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(RuntimeError): ray::TuneModel.train() (pid=58322, ip=10.44.1.111, actor_id=7a19d60d9898f6c32b0c1a0201000000, repr=<src.learner.raytune_learner.TuneModel object at 0x7f3ac830c460>)
  File "/home/luis

(TuneModel pid=58322) torch.Size([7, 13780]) [repeated 2x across cluster]


2024-05-06 17:58:32,230	ERROR tune_controller.py:1383 -- Trial task failed for trial TuneModel_76f62_00001
Traceback (most recent call last):
  File "/home/luisasantus/mambaforge/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/luisasantus/mambaforge/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/luisasantus/mambaforge/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/home/luisasantus/mambaforge/lib/python3.10/site-packages/ray/_private/worker.py", line 2563, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(RuntimeError): ray::TuneModel.train() (pid=58321, ip=10.44.1.111, actor_id=110b38b3aff1af874347bb9901000000, repr=<src.learner.raytune_learner.TuneModel object at 0x7f30382f03d0>)
  File "/home/luis

(TuneModel pid=58796) torch.Size([7, 13780]) [repeated 16x across cluster]


2024-05-06 17:58:39,229	ERROR tune_controller.py:1383 -- Trial task failed for trial TuneModel_76f62_00009
Traceback (most recent call last):
  File "/home/luisasantus/mambaforge/lib/python3.10/site-packages/ray/air/execution/_internal/event_manager.py", line 110, in resolve_future
    result = ray.get(future)
  File "/home/luisasantus/mambaforge/lib/python3.10/site-packages/ray/_private/auto_init_hook.py", line 24, in auto_init_wrapper
    return fn(*args, **kwargs)
  File "/home/luisasantus/mambaforge/lib/python3.10/site-packages/ray/_private/client_mode_hook.py", line 103, in wrapper
    return func(*args, **kwargs)
  File "/home/luisasantus/mambaforge/lib/python3.10/site-packages/ray/_private/worker.py", line 2563, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(RuntimeError): ray::TuneModel.train() (pid=58828, ip=10.44.1.111, actor_id=405e1213f884bdca1214652701000000, repr=<src.learner.raytune_learner.TuneModel object at 0x7f4b21488490>)
  File "/home/luis

RuntimeError: No best trial found for the given metric: val_loss. This means that no trial has reported this metric, or all values reported for this metric are NaN. To not ignore NaN values, you can set the `filter_nan_and_inf` arg to False.

In [15]:
tuner = learner.tuner
tuner